In [ ]:
import numpy as np
import scipy.io
from midiutil import MIDIFile
from utils import *

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

#keys = scipy.io.loadmat('MATLAB_data/31Jan_jc_upper_octave_noise.mat')['key']
keys = scipy.io.loadmat('MATLAB_data/jc_upper_octave.mat')['key']
notes_list = scipy.io.loadmat('notes_100.mat')['notes']
freq_list = scipy.io.loadmat('freq_480_octave_down.mat')['f_cropped']
# notes_list = scipy.io.loadmat('notes_88.mat')['notes']
# freq_list = scipy.io.loadmat('freq_480.mat')['f_cropped']
#freq_list = scipy.io.loadmat('freq_441.mat')['f_cropped']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 100 #88+12, 12 from upper octave

In [ ]:
notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key,cur_freq,notes,notes_list)

print(len(notes))

In [ ]:
notes = sorted(notes, key = lambda note: note.start)
notes = sorted(notes, key = lambda note: -note.pitch)

In [ ]:
for note in notes:
    print(note)

In [ ]:
new_notes = remove_repetitive(notes,note_num)
for note in new_notes:
    print(note)

In [ ]:
print(len(new_notes))

In [ ]:
nnew_notes = multichannel(new_notes,note_num)
for note in nnew_notes:
    print(note)

In [ ]:
import matplotlib.pyplot as plt
def plot_notes(note_list):
    time_lower = []
    pitch_lower = []
    time_upper = []
    pitch_upper = []
    for note in note_list:
        if note.pitch < 89:
            time_lower.append(note.start)
            pitch_lower.append(note.pitch)
        else:
            time_upper.append(note.start)
            pitch_upper.append(note.pitch)
        
    plt.scatter(time_upper,pitch_upper,s=1)
    plt.scatter(time_lower,pitch_lower,s=1)

In [ ]:
def plot_notes(note_list):
    time =[]
    pitch = []
    for note in note_list:
        time.append(note.start)
        pitch.append(note.pitch)
    plt.scatter(time,pitch,s=1)

In [ ]:
plot_notes(new_notes)

In [ ]:
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, 1200)

In [ ]:
for note in nnew_notes:
    v = int(50+note.sign()*12.5)
    mf.addNote(track,note.channel,note.pitch+20, note.start, note.end-note.start+1,v)

In [ ]:
with open("midifile/jc_upper_octave_.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
# print(len(nnew_notes))
# print(len(notes))

In [ ]:
# def picknotes_old(cur_key,cur_freq):
#     j = 0
#     while j < len(cur_key):
#         if cur_key[j] != 0:
            
#             d = {}
#             d['note'] = cur_key[j]
#             d['start'] = j
#             while cur_key[j] != 0 and j < len(cur_key)-1:
#                 j += 1
#             d['end'] = j - 1
#             intensity = get_intensity(cur_key[j-1],cur_freq,notes_list[cur_key[j-1]-1])
#             d['intensity'] = intensity
#             notes.append(d)
#         j += 1